In [1]:
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from vol.vol import Heston

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg, european_call_payoff
from hestonmc_cuda import mc_price_cupy, simulate_heston_euler_cupy, simulate_heston_andersen_qe_cupy, simulate_heston_andersen_tg_cupy, european_call_payoff_cupy

In [2]:
#heston_params = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3) #from stoch vol
#heston_params = HestonParameters(kappa = 1, gamma = 0.4, rho = -0.1, vbar = 0.2, v0 = 0.2) # from school
#heston_params = HestonParameters(kappa = 0.5, gamma = 1, rho = -0.9, vbar = 0.04, v0 = 0.04) #  from andeson paper 1
heston_params = HestonParameters(kappa = 0.3, gamma = 0.9, rho = -0.5, vbar = 0.04, v0 = 0.04) #  from andeson paper 2
#heston_params = HestonParameters(kappa = 1, gamma = 1, rho = -0.3, vbar = 0.04, v0 = 0.09) #  from andeson paper 3
 
state         = MarketState(stock_price = 1.*100, interest_rate = 0.)
model         = Heston(state.stock_price, heston_params.v0, heston_params.kappa, heston_params.vbar, heston_params.gamma, heston_params.rho, state.interest_rate)

r_x           = np.load(r"Data/anderson tg/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = np.load(r"Data/anderson tg/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = np.load(r"Data/anderson tg/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
kwargs        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }
r_x           = cp.load(r"Data/anderson tg/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = cp.load(r"Data/anderson tg/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = cp.load(r"Data/anderson tg/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
kwargs_cupy        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }

In [3]:
import time
from tqdm.contrib.itertools import product

In [ ]:
test_1 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(70, 120, 70),
            'Ts': np.linspace(0.1, 5, 70),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(100_000, 1000_000, 100000_000),
            'heston_params': heston_params_1

In [7]:
heston_params_1 = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3) #from stoch vol
#heston_params_2 = HestonParameters(kappa = 1, gamma = 0.4, rho = -0.1, vbar = 0.2, v0 = 0.2) # from school
#heston_params_3 = HestonParameters(kappa = 0.5, gamma = 1, rho = -0.9, vbar = 0.04, v0 = 0.04) #  from andeson paper 1
#heston_params_4 = HestonParameters(kappa = 0.3, gamma = 0.9, rho = -0.5, vbar = 0.04, v0 = 0.04) #  from andeson paper 2
#heston_params_5 = HestonParameters(kappa = 1, gamma = 1, rho = -0.3, vbar = 0.04, v0 = 0.09) #  from andeson paper 3

state         = MarketState(stock_price = 1.*100, interest_rate = 0.)


test_1 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(70, 120, 70),
            'Ts': np.linspace(0.1, 5, 70),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(100_000, 1000_000, 100000_000),
            'heston_params': heston_params_1

        }
MC_compare_models_grid_test_1= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])


i = 0

for scheme, strike, T, N_T, batch_size in product(test_1['schemes'], test_1['strikes'], test_1['Ts'], test_1['N_Ts'], test_1['batch_sizes']):
    ec_payoff = european_call_payoff_cupy(T, strike, state.interest_rate)
    
    common_mc_params = {"absolute_error": 5e-2, "state": state, "heston_params": test_1['heston_params'], "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": False}
    model = Heston(state.stock_price, heston_params.v0, heston_params.kappa, heston_params.vbar, heston_params.gamma, heston_params.rho, state.interest_rate)
    
    if scheme == simulate_heston_andersen_tg_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params, **kwargs_cupy))
        et = time.time()
    elif scheme == simulate_heston_euler_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params))
        et = time.time()
    elif scheme == simulate_heston_andersen_qe_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params))
        et = time.time()
    MC_compare_models_grid_test_1.loc[i] = (scheme.__name__, 1, strike, T, N_T, batch_size,common_mc_params['absolute_error'], model.call_price(T, strike), res, et-st)
    i +=1

  0%|          | 0/19600 [00:00<?, ?it/s]

In [131]:
MC_compare_models_grid_test_1  = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_1_fixed_true.csv', index_col=0)

In [119]:
#MC_compare_models_grid_test_1.to_csv(r"Data/evaluation/MC_compare_models_grid_test_1_fixed_true.csv")

In [132]:
MC_compare_models_grid_test_1

,scheme,heston_params#,strike,T,N_T,batch_size,absolute error,true,MC_price,time
0,simulate_heston_euler_cupy,1,70.0,0.100000,50,100000,0.05,30.131379,30.129462,0.111071
1,simulate_heston_euler_cupy,1,70.0,0.100000,100,100000,0.05,30.131379,30.143064,0.198346
2,simulate_heston_euler_cupy,1,70.0,0.171014,50,100000,0.05,30.499960,30.483909,0.145198
3,simulate_heston_euler_cupy,1,70.0,0.171014,100,100000,0.05,30.499960,30.480420,0.301842
4,simulate_heston_euler_cupy,1,70.0,0.242029,50,100000,0.05,30.955468,30.929116,0.180836
...,...,...,...,...,...,...,...,...,...,...
19595,simulate_heston_andersen_tg_cupy,1,120.0,4.857971,100,100000,0.05,18.684402,18.809335,1.729174
19596,simulate_heston_andersen_tg_cupy,1,120.0,4.928986,50,100000,0.05,18.811424,19.045366,0.698318
19597,simulate_heston_andersen_tg_cupy,1,120.0,4.928986,100,100000,0.05,18.811424,18.934070,1.756082
19598,simulate_heston_andersen_tg_cupy,1,120.0,5.000000,50,100000,0.05,18.937836,19.157635,0.713773


In [142]:
test2 = MC_compare_models_grid_test_1[MC_compare_models_grid_test_1['scheme'] == 'simulate_heston_euler_cupy'][MC_compare_models_grid_test_1['N_T'] == 50][['strike', 'T', 'MC_price', 'true']]

/tmp/ipykernel_12715/4027686923.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [143]:
test2['error'] = test2['MC_price'] - test2['true']

In [145]:
test2 = test2.pivot(index='strike', columns='T', values='error')

In [128]:
test2 = test2.pivot(index='strike', columns='T', values='true')

In [94]:
test2 = test2.pivot(index='strike', columns='T', values='MC_price')

In [147]:
import plotly.graph_objects as go
import plotly.express as px

"""x= np.linspace(0.1, 1.1, 0.1)
y= np.linspace(85, 120, 5)"""
fig = go.Figure(data=[go.Surface(z=test2.values, x=test2.columns, y=test2.index)])
fig.show()

In [106]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure(data=[go.Surface(z=test2.values, x=test2.columns, y=test2.index)])
fig.show()